## Import data from csv

- Currently using downloaded source data from grab website
- To create a function that reads in zip before submission
- Link to zip download here: "https://s3-ap-southeast-1.amazonaws.com/grab-aiforsea-dataset/safety.zip" 

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df0 = pd.read_csv("../grab-ai-safety-data/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df1 = pd.read_csv("../grab-ai-safety-data/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df2 = pd.read_csv("../grab-ai-safety-data/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df3 = pd.read_csv("../grab-ai-safety-data/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df4 = pd.read_csv("../grab-ai-safety-data/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df5 = pd.read_csv("../grab-ai-safety-data/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df6 = pd.read_csv("../grab-ai-safety-data/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df7 = pd.read_csv("../grab-ai-safety-data/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df8 = pd.read_csv("../grab-ai-safety-data/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df9 = pd.read_csv("../grab-ai-safety-data/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
response = pd.read_csv("../grab-ai-safety-data/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [3]:
df_features = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0)
response = response.drop_duplicates(subset="bookingID", keep=False)

In [4]:
df = pd.merge(
    df_features,
    response,
    how="left",
    on="bookingID"
).sort_values(
    ["bookingID", "second"],
    ascending=True
)

df.columns = [x.lower() for x in df.columns]

In [5]:
with open('../grab-ai-safety-data/df_full.pickle', 'wb') as f:
    pickle.dump(df, f)